<a href="https://colab.research.google.com/github/shashigharti/secure-privateai-scholarship-challenge/blob/master/%5BPytorch%5DNN_from_scratch%2C_Cat_Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 78.1MB/s]
test1.zip: Skipping, found more recently modified local copy (use --force to force download)
 99% 540M/543M [00:04<00:00, 50.8MB/s]
100% 543M/543M [00:04<00:00, 136MB/s] 


In [0]:
!rm -rf /content/data
!rm -rf /content/test1
!rm -rf /content/train

In [0]:
!mkdir data
!mkdir /content/data/Cat_Dog_data
!mv /content/train.zip /content/data/
!mv /content/sampleSubmission.csv /content/data/

In [0]:
!unzip /content/data/train.zip -d /content/data/

In [0]:
# check
!find /content/data/train/ -type f -name 'cat*' | wc -l # 12500
!find /content/data/train/ -type f -name 'dog*' | wc -l # 12500

12500
12500


In [0]:
#Make cat dog directories
!mkdir -p /content/data/Cat_Dog_data/train/dog
!mkdir -p /content/data/Cat_Dog_data/train/cat
!mkdir -p /content/data/Cat_Dog_data/val/dog
!mkdir -p /content/data/Cat_Dog_data/val/cat

In [0]:
!find /content/data/train -name 'cat*' -type f | shuf -n12500 | xargs -I file mv file /content/data/Cat_Dog_data/train/cat
!find /content/data/train -name 'cat*' -type f | xargs -I file mv file /content/data/Cat_Dog_data/val/cat

In [0]:
!find /content/data/train -name 'dog*' -type f | shuf -n12500 | xargs -I file mv file /content/data/Cat_Dog_data/train/dog
!find /content/data/train -type f -name 'dog*' | xargs -I file mv file /content/data/Cat_Dog_data/val/dog

In [0]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import torch.nn.functional as F
import helper

In [0]:
#set data directory
data_dir = '/content/data/Cat_Dog_data'
train_dir = f'{data_dir}/train'
validation_dir = f'{data_dir}/val'

In [0]:
#create a new transform
train_transforms = transforms.Compose([
    transforms.RandomRotation(30), # do random rotation
    transforms.RandomResizedCrop(224), #crop image size
    transforms.RandomHorizontalFlip(), # add horizontal flip
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

train_dataset = datasets.ImageFolder(train_dir, transform = train_transforms)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = 64, shuffle = True)

validation_dataset = datasets.ImageFolder(validation_dir, transform = test_transforms)
validationloader = torch.utils.data.DataLoader(validation_dataset, batch_size = 64, shuffle = True)

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

In [0]:
images, labels = next(iter(trainloader))
fig, axes = plt.subplots(figsize=(15,15), ncols=5)
for ii in range(5):
  helper.imshow(images[ii], ax=axes[ii], normalize=False)


In [0]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    
    self.fc1 = nn.Linear(150528, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 64)
    self.fc4 = nn.Linear(64, 10)
    
    self.dropout = nn.Dropout(p=0.2)
    
  def forward(self, x):
    #Flatten the image
    x = x.view(x.shape[0], -1)
       
    
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))
    x = F.log_softmax(self.fc4(x), dim = 1)
    
    return x
model = Network()

In [0]:
epochs = 5
optimizer = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.NLLLoss()

validation_losses, training_losses = [],[]
for epoch in range(epochs):
  training_loss = 0
  validation_loss = 0
  accuracy = 0
  
  for images, labels in trainloader:
    optimizer.zero_grad() #donot remember previous grad
    log_ps = model(images)
    loss = criterion(log_ps, labels)
    loss.backward() #calculate gradient
    optimizer.step() #update weights
    
    training_loss += loss.item()
  print("Training Loss:{:.3f}", (training_loss/len(trainloader)))
    
    
    



In [0]:
#obtain training indices that will be used for  validation
num_train = len(train_dataset)
#get the indices
indices = list(range(num_train))